## Module 1

### Part-1 Embeddings & pgVector

- Understand the Vector Embeddings
- How to store vector data inside Aurora PostgreSQL.
- Describe the importance of tuning the pgvector index and search best practices


**Note**

To keep things simple, we have put together multiple utility functions. You do not have to learn the code in these utility functions (unless you want to :). Do run the cells with utility functions and start from cell # 1.

In [ ]:
## Install the driver for PostgreSQL

!pip install "psycopg[binary]"

## Create DB connection

* Retrieve the database credentials from secrets manager
* Setup database connection

In [ ]:
import psycopg
from psycopg2 import sql 
import boto3
import json
import sys
import time

sm_client = boto3.client('secretsmanager')

response = sm_client.get_secret_value(SecretId='apgpg-pgvector-secret')
database_secrets = json.loads(response['SecretString'])

dbhost = database_secrets['host']
dbport = database_secrets['port']
dbuser = database_secrets['username']
dbpass = database_secrets['password']

dbconn = psycopg.connect(host=dbhost, user=dbuser, password=dbpass, port=dbport, connect_timeout=10, autocommit=True)

## Utility functions

### Database Utility functions

In [ ]:
# Invoke a SQL statement
# Close the cursor after you are done with the call
def invoke_sql(statement):
    cur = dbconn.cursor()
    try:
        cur.execute(statement)
        return cur
    except Exception as error:
        print("DB statement execution error !!!", error)
        sys.exit(1)

def invoke_sql_dump_rows(statement, all=True):
    cur = invoke_sql(statement)

    if all:
        rows = cur.fetchall()

        for row in rows:
            print(row)
    else:
        
        row = cur.fetchone()
        print(row)
        
    cur.close()
    
def invoke_sql_dml(statement):
    cur = dbconn.cursor()
    try:
        cur.execute(statement)
        cur.close()
    except Exception as error:
        print("DB statement execution error !!!", error)
        sys.exit(1)

# Create table with a vector attribute
# NO action taken, if the table {table_name} already exist
# Otherwise creates the table with following attributes:  id(TEXT),  chunk(TEXT),   embedding(VECTOR[vector_dim])   metadata(JSONB)
def create_test_table(table_name, vector_dim):
    vector_dim = int(vector_dim)
    cur = dbconn.cursor()

    cur.execute(f"""
        SELECT EXISTS (
            SELECT 1
            FROM information_schema.tables
            WHERE table_schema = 'public' AND table_name = '{table_name}'
        );
        """)
    exists = cur.fetchone()[0]

    if exists:
        print(f"Table '{table_name}' already exists.")
        # truncate the table
        invoke_sql_dml(f"TRUNCATE TABLE {table_name}")
    else: 
        try:
            cur.execute(f"""
                    CREATE TABLE {table_name} (
                        _id SERIAL PRIMARY KEY,
                        text TEXT,
                        embedding VECTOR({vector_dim})
                    );
                """)
            print("Created the table : ", table_name)
        except Exception as e:
            dbconn.rollback()
            print(f"An error occurred while creating the table: {e}")

    dbconn.commit()
    cur.close()

    
# Create the product table
def create_product_table(product_table_name, vector_dim):
    vector_dim = int(vector_dim)
    cur = dbconn.cursor()

    cur.execute(f"""
        SELECT EXISTS (
            SELECT 1
            FROM information_schema.tables
            WHERE table_schema = 'public' AND table_name = '{product_table_name}'
        );
        """)
    exists = cur.fetchone()[0]

    if exists:
        print(f"Table '{product_table_name}' already exists.")
        # truncate the table
        invoke_sql_dml(f"TRUNCATE TABLE {product_table_name}")
    else: 
        try:
            cur.execute(f"""
                    CREATE TABLE {product_table_name} (
                        id TEXT PRIMARY KEY,
                        text TEXT,
                        embedding VECTOR({vector_dim}),
                        product_image  TEXT,
                        metadata  JSONB
                    );
                """)
            print("Created the table : ", product_table_name)
        except Exception as e:
            dbconn.rollback()
            print(f"An error occurred while creating the table: {e}")

    dbconn.commit()
    cur.close()
    
# Create product table & populate it with products data available in local JSONL file
# Intent is just to show the code that was used for populating the product table
def   create_and_populate_product_table(product_table_name, vector_dimension):
    
    # Creates the product table
    create_product_table(product_table_name, vector_dimension)
    
    # Product data file
    file_path = "./data/amazon.jsonl"
    
    # Read 1 product at a time, generate embedding and insert to product table
    with open(file_path, 'r') as file:
            for line in file:
                data = json.loads(line)
                # Process the JSON object (data) here
                input_text = data['category'] + "\n" + data["product_name"] + "\n"
                embedding_input_text = create_embedding(input_text)
                product_image = data['product_image']
                # print(input_text)
                id = data['id']
                metadata = {}
                sql = f"INSERT INTO {product_table_name} (id, embedding, text, metadata, product_image) VALUES ('{id}', '{embedding_input_text}', '{input_text}', '{metadata}', '{product_image}')"
                # print(sql)
                invoke_sql_dml(sql)
                print(".", end='')
    
    
# Utility function to display the image for products
def invoke_sql_dump_rows_with_images(sql):
    cur = invoke_sql(sql)
    rows = cur.fetchall()
    for row in rows:
        print(row[1])
        print(row[2])
        url = row[2]
        i = Image(url=url, width=200)
        display(i)
    cur.close()
    


### Plot bar chart
Function to draw a plot.

In [ ]:
import matplotlib.pyplot as plt

def  plot_bar_chart(categories, values, title):
    # Create the bar chart
    plt.bar(categories, values)

    plt.title(title)

    # Display the chart
    plt.show()

### Embedding utility function

This function will use Amazon Titan Text Embeddings modeel to generate embeddings.

![embeddings|100x100,100%](images/gen_embeddings.png)

https://docs.aws.amazon.com/code-library/latest/ug/python_3_bedrock-runtime_code_examples.html

In [ ]:

bedrock_client = boto3.client("bedrock-runtime")

# Each model supprts discrete vector dimensions. Model we are using supports vector dimension of 1024
model_id = "amazon.titan-embed-text-v2:0"
vector_dimension = 1024

def create_embedding(input_text):
    # Create the request for the model.
    native_request = {"inputText": input_text}
    
    # Convert the native request to JSON.
    request = json.dumps(native_request)
    
    # Invoke the model with the request.
    response = bedrock_client.invoke_model(modelId=model_id, body=request)
    
    # Decode the model's native response body.
    model_response = json.loads(response["body"].read())

    # Extract and print the generated embedding and the input text token count.
    embedding = model_response["embedding"]
    
    return embedding



## 1. Verify that pgvector extension is installed

pgvector is an open-source extension for PostgreSQL, which allows you to store vector embeddings and perform vector similarity searches in Postgres."

In [ ]:
# Invoke a SQL to get a list of installed extensions and their versions
# pg_vector extention is already installed in the PostgreSQL database.

sql = "SELECT oid, extname, extversion FROM pg_extension"

invoke_sql_dump_rows(sql)

## 2. Create a table with attribute of type vector

1. Creates a table if it does NOT exist. The table has a *embedding* attribute that holds a vector of pre-defined size i.e., specified vector dimension.

2. Get the DDL for the table

* _id           PK 
* chunk         Text    :   This is the actual text that is embedded
* embedding     Vector(dimension)  :  Vector representation for the text chunk
* metadata      JSONB



In [ ]:
# Table name - change the names as needed
table_name = f"table_test_1_dim_{vector_dimension}"

# Create table if it doesn't exist
create_test_table(table_name, vector_dimension)

sql = f"SELECT a.attname, pg_catalog.format_type(a.atttypid, a.atttypmod)FROM pg_catalog.pg_attribute a \
        WHERE a.attrelid = (SELECT c.oid FROM pg_catalog.pg_class c \
        LEFT JOIN pg_catalog.pg_namespace n ON n.oid = c.relnamespace WHERE c.relname='{table_name}')   AND a.attnum > 0 ;"
invoke_sql_dump_rows(sql)

## 3. Create test data embeddings and add to the table

These are text input examples to demonstrate how vectors are added and queried in PostgreSQL with the pgvector extension.

**NOTE:** Running this multple times will result in the row getting added multiple times !!

In [ ]:
# Test data - Food, Sports
input_data = [
    "I love baseball",
    "I love pizza",
    "my favorite sports is football",
    "my favorite food is pasta",
    "I like enchiladas",
    "I cant miss the basketball match"
]

# Generate the embedding for data and insert to the table
for text in input_data:
    embedding = create_embedding(text)
    sql = f"INSERT INTO {table_name} (text, embedding) VALUES ('{text}', '{embedding}');"
    invoke_sql_dml(sql)

# Dump the count of rows in the table
invoke_sql_dump_rows(f"SELECT count(*) FROM {table_name}")

## 4. Semantic search without Indexing i.e., exact search

In the cell below, we have given you sample queries in a list, change the queries[x] index to search given string.
ex: queries[0], will search "I like mexican food"


In [ ]:
# sample queries
queries = [
    "I like mexican food",
    "I like italian food",
    "What is your favorite food",
    "I am a sports fan",
]

# Change index to try out different queries, try any value between 0 to 3
query = queries[0] 

# Generate the embedding for the query
query_embedding = create_embedding(query)

# Do an exact search with kNN using the L2 distance operator '<->'
# You may try different operators, the results won't change 

###  <#> - (negative) inner product    
###  <=> - cosine distance
###  <+> - L1 distance (added in 0.7.0) 
###  <~> - Hamming distance (binary vectors, added in 0.7.0)
###. <%> - Jaccard distance (binary vectors, added in 0.7.0)

# Set the value of k, this is number of records to retrive.
k = 3

# Check out the query to retrieve the semantically closest text from the table
sql = f"SELECT text FROM {table_name} ORDER BY embedding <-> '{query_embedding}' LIMIT {k};"
invoke_sql_dump_rows(sql)

## 5. Prepare, Load product data and create index
So far we have worked with a small number of rows and indexing does not really provides us any benefit. But as the number of rows increases we will start to see the benefit of ANN algorithms. 

In order to save time, we have already created the product taable and populated it with roughly 10K products. We will query this table with kNN exact search and then create an index of carrying out ANN search.

If yoou are curious, how we populated the table. Here is the code:

```
# Code for populating the product table
vector_dimension=1024
product_table_name = f"product_table_5_dim_{vector_dimension}"

create_and_populate_product_table(product_table_name, vector_dimension)
```

In [ ]:
# The product table is already created and populated
product_table_name = f"product_table_dim_{vector_dimension}"

# Check the structure of product table
sql = f"SELECT a.attname, pg_catalog.format_type(a.atttypid, a.atttypmod)FROM pg_catalog.pg_attribute a \
        WHERE a.attrelid = (SELECT c.oid FROM pg_catalog.pg_class c \
       LEFT JOIN pg_catalog.pg_namespace n ON n.oid = c.relnamespace WHERE c.relname='{product_table_name}')   AND a.attnum > 0 ;"

print("Product table structure")
print("=======================")
invoke_sql_dump_rows(sql)

# Dump the count of rows in the table
print("\nProduct table row count")
print("=======================")
invoke_sql_dump_rows(f"SELECT count(*) FROM {product_table_name}")

### Setup search query & query string embedding : Product table

In [ ]:

from IPython.display import Image, display
from urllib.request import urlopen

# Try out the following queries, uncomment one query at a time.
query = "I am looking for a toy that can teach language to children"
#query = "I am looking for a toy that can teach music to children"
#query = "I am looking for a toy that can teach crafts to children"
#query = "I am looking for a toy that can teach knitting to children"
#query = "I am looking for a toy that can teach engineering to children"

# Set the number of records to be fetched
k=5

# Create the embedding for the query string
query_embedding = create_embedding(query)



### Run exact search : Product table

In [ ]:
%%time

# L2 distance operator
sql = f"SELECT id, text, product_image FROM {product_table_name} ORDER BY embedding <-> '{query_embedding}' LIMIT {k};"
# invoke_sql_dump_rows(sql)

invoke_sql_dump_rows_with_images(sql)

### 6. HNSW Index creation & ANN

Let's create an index and try out ANN. Pay attention to the distance operator !!!

* **m**  (between 2 & 100, default = 16) : the max number of connections per layer. A larger M means a more interconnected, denser graph but one that will consume more memory and be slower to insert.

* **ef_construction** (between 4 & 1000, default = 64): the size of the dynamic candidate list for constructing the graph (64 by default). Bigger ef_construction leads to longer construction, but better index quality. ef_construction must be greater than or equal to 2 * m

After running the ANN search compare the query latency for exact vs ANN search. You will find the runtime stats after the query results.

In [ ]:
# Create the index

sql = f"CREATE INDEX ON {product_table_name} USING hnsw (embedding vector_cosine_ops) WITH (m=32, ef_construction=128)"

invoke_sql_dml(sql)

### Run ANN search : Product table

In [ ]:
%%time

# Run ANN

# Cosine distance operator
# Compare this query with the exact search query - ONLY difference is the distance operator !!!
sql = f"SELECT id, text, product_image FROM {product_table_name} ORDER BY embedding <=> '{query_embedding}' LIMIT {k};"

invoke_sql_dump_rows_with_images(sql)

### References

[pgVector on GitHub](https://github.com/pgvector/pgvector)

[pgVector v0.70](https://www.postgresql.org/about/news/pgvector-070-released-2852/)

[Effect of Quantization](https://jkatz05.com/post/postgres/pgvector-scalar-binary-quantization/)